In [1]:
############################################################################################################
## 1. 패키지 import
############################################################################################################
import pandas as pd
import sqlite3
import os
from datetime import datetime, timedelta
import time
from dateutil.relativedelta import relativedelta

# 사용 예시
from module.logger import log_message
import module.sqlTransaction as sqlT
import importlib
importlib.reload(sqlT)

############################################################################################################
## 2. 초기설정
############################################################################################################
# 경로설정
dir_work   = f'c:/Users/user/OneDrive - 파인트리파트너스(주)/movie'
dir_func   = f'{dir_work}/src/module'
dir_data   = f'{dir_work}/data'

# DB연결
conn = sqlite3.connect(f"{dir_data}/pine_movie.db", isolation_level=None)
cur = conn.cursor()

# 함수 호출
# exec(open(f"{dir_func}/sqlTransaction.py"      , encoding= 'utf-8').read() )
# exec(open(f"{dir_func}/logger.py"              , encoding= 'utf-8').read() )

# EDA 개요
###  ●  모델의 목적 : 회원별 구매 가능성이 높은 장르를 추천하는 모델 

##### ----- 추천에 적합한 장르를 선정
##### ----- 장르 선호 이벤트를 정의


# 데이터 추출

In [2]:
############################################################################################################
## 전체 테이블 데이터 추출
############################################################################################################
df_cast                 = pd.read_sql(f"""select * from CAST              """ , conn)
df_crew                 = pd.read_sql(f"""select * from CREW              """ , conn)
df_cust                 = pd.read_sql(f"""select * from CUSTOMER          """ , conn)
df_cus_seg              = pd.read_sql(f"""select * from CUSTOMER_SERGMENT """ , conn)
df_genre                = pd.read_sql(f"""select * from GENRE             """ , conn)
df_mov                  = pd.read_sql(f"""select * from MOVIE             """ , conn)
df_mov_cast             = pd.read_sql(f"""select * from MOVIE_CAST        """ , conn)
df_mov_crew             = pd.read_sql(f"""select * from MOVIE_CREW        """ , conn)
df_mov_fact             = pd.read_sql(f"""select * from MOVIE_FACT        """ , conn)
df_mov_genre            = pd.read_sql(f"""select * from MOVIE_GENRE       """ , conn)
df_actv                 = pd.read_sql(f"""select * from ACTIVITY          """ , conn)


# 이벤트 정의 - 회원행동 로그 기반

In [7]:
# cur.execute(f"""DROP TABLE CUST_ACT""")
cur.execute(
 f""" 
      CREATE TABLE IF NOT EXISTS CUST_ACT AS
      SELECT 
          a01.회원번호
        , a01.기준년월
        , a01.영화번호
        , max(case when a01.행동번호 = 5   then 1        else 0 end)          AS 탐색여부
        , max(case when a01.행동번호 = 11  then 1        else 0 end)          AS 구매여부
        , max(case when a01.행동번호 = 4   then 1        else 0 end)          AS 시작여부
        , max(case when a01.행동번호 = 2   then 1        else 0 end)          AS 완료여부
        , max(case when a01.행동번호 = 1   then 1        else 0 end)          AS 평점여부
      FROM (
         SELECT
                  a01.회원번호
                , a01.기준년월
                , a01.기준년월일
                , a01.영화번호
                , a01.행동번호
                , a01.평점
                , a01.판매액
        FROM MOVIE_FACT a01
        GROUP BY a01.기준년월, a01.회원번호, a01.영화번호, a01.행동번호
      ) a01
    
      LEFT JOIN movie_fact a02
      ON 
          a01.기준년월 = a02.기준년월 
      AND a01.회원번호 = a02.회원번호 
      AND a01.영화번호 = a02.영화번호 
    
      GROUP BY a02.기준년월, a02.회원번호, a02.영화번호
 """ 
)
pd.read_sql(f" select * from CUST_ACT", conn)


,회원번호,기준년월,영화번호,탐색여부,구매여부,시작여부,완료여부,평점여부
0,1000083,201001,2255,0,0,1,0,1
1,1000083,201001,14160,1,0,0,0,0
2,1000104,201001,279,1,0,0,0,0
3,1000104,201001,14168,1,0,0,0,0
4,1000186,201001,1245,1,0,0,0,0
...,...,...,...,...,...,...,...,...
1694943,1432271,201210,94730,0,1,0,1,0
1694944,1432271,201210,1131183,0,0,0,0,1
1694945,1445052,201210,829,1,1,0,1,0
1694946,1445052,201210,27205,0,1,0,0,0


In [9]:
pd.read_sql(
 f""" 
    SELECT
            행동패턴
          , count(1)                            AS 건수
    FROM (
      SELECT
            구매여부 || 시작여부 || 완료여부 || 평점여부                   as 행동패턴
      FROM CUST_ACT 
    )
    GROUP BY 행동패턴
    ORDER BY 행동패턴 DESC
 """ 
, conn)

,행동패턴,건수
0,1111,90290
1,1110,344726
2,1100,2
3,1011,15
4,1010,1328
5,1001,12
6,1000,776
7,0111,165
8,0110,633
9,0101,10473


In [10]:
df_event = pd.read_sql(
 f""" 
    SELECT 
          기준년월 
        , SUM(건수)                                              AS 전체건수
        , SUM(CASE WHEN 행동패턴 <> '0000' THEN 건수 ELSE 0 END) AS 이벤트건수
    FROM (
        SELECT
              기준년월
            , 행동패턴
            , count(1)                            AS 건수
        FROM (
            SELECT
                  기준년월
                , 구매여부 || 시작여부 || 완료여부 || 평점여부                   as 행동패턴
            FROM CUST_ACT 
        )
        GROUP BY 기준년월, 행동패턴
    )
    GROUP BY 기준년월
 """ 
, conn)

In [11]:
df_event['이벤트비율'] = df_event['이벤트건수'] / df_event['전체건수']
df_event

,기준년월,전체건수,이벤트건수,이벤트비율
0,201001,47399,18198,0.383932
1,201002,36955,13932,0.376999
2,201003,40675,15486,0.380725
3,201004,37080,14161,0.381904
4,201005,43283,16756,0.387127
5,201006,39258,15007,0.382266
6,201007,40891,15707,0.384119
7,201008,40401,15681,0.388134
8,201009,39288,15119,0.384825
9,201010,44868,17430,0.388473


# 장르 선정 

### 1. 장르컬럼에 대한 상세 탐색

In [12]:
######################################################################
### 1-1. 장르별 영화건수
######################################################################
pd.read_sql(
 f""" 
     SELECT
            t1.장르번호
          , t2.장르명
          , COUNT(*)                                     as 장르건수
     FROM movie_genre t1

     LEFT JOIN genre t2
     ON t2.장르번호 = t1.장르번호

     GROUP BY t1.장르번호
     ORDER BY 장르건수 desc
 """ , conn
) 


,장르번호,장르명,장르건수
0,3,Drama,2272
1,6,Comedy,1742
2,9,Thriller,771
3,15,Romance,657
4,17,Horror,606
5,7,Action,591
6,45,Sci-Fi,537
7,8,Crime,517
8,12,Fantasy,504
9,14,Family,437


In [13]:
######################################################################
### 1-2. 영화 선호 패턴 건수가 적은 장르
######################################################################
pd.read_sql(
    f"""
        SELECT
             기준년월
            , SUM(CASE WHEN 장르번호 = 1                  THEN 1 ELSE 0 END )                 AS 역사
            , SUM(CASE WHEN 장르번호 = 2                  THEN 1 ELSE 0 END )                 AS 애니메이션
            , SUM(CASE WHEN 장르번호 = 3                  THEN 1 ELSE 0 END )                 AS 드라마
            , SUM(CASE WHEN 장르번호 = 6                  THEN 1 ELSE 0 END )                 AS 코미디
            , SUM(CASE WHEN 장르번호 = 7                  THEN 1 ELSE 0 END )                 AS 액션
            , SUM(CASE WHEN 장르번호 = 8                  THEN 1 ELSE 0 END )                 AS 범죄
            , SUM(CASE WHEN 장르번호 = 9                  THEN 1 ELSE 0 END )                 AS 스릴러
            , SUM(CASE WHEN 장르번호 = 10                 THEN 1 ELSE 0 END )                 AS 다큐멘터리
            , SUM(CASE WHEN 장르번호 = 11                 THEN 1 ELSE 0 END )                 AS 모험
            , SUM(CASE WHEN 장르번호 = 12                 THEN 1 ELSE 0 END )                 AS 판타지
            , SUM(CASE WHEN 장르번호 = 14                 THEN 1 ELSE 0 END )                 AS 가족
            , SUM(CASE WHEN 장르번호 = 15                 THEN 1 ELSE 0 END )                 AS 로맨스
            , SUM(CASE WHEN 장르번호 = 16                 THEN 1 ELSE 0 END )                 AS 뮤직
            , SUM(CASE WHEN 장르번호 = 17                 THEN 1 ELSE 0 END )                 AS 공포
            , SUM(CASE WHEN 장르번호 = 18                 THEN 1 ELSE 0 END )                 AS 전쟁
            , SUM(CASE WHEN 장르번호 = 19                 THEN 1 ELSE 0 END )                 AS 서부극
            , SUM(CASE WHEN 장르번호 = 20                 THEN 1 ELSE 0 END )                 AS 미스터리
            , SUM(CASE WHEN 장르번호 = 24                 THEN 1 ELSE 0 END )                 AS 단극
            , SUM(CASE WHEN 장르번호 = 25                 THEN 1 ELSE 0 END )                 AS 뮤지컬
            , SUM(CASE WHEN 장르번호 = 30                 THEN 1 ELSE 0 END )                 AS 스포츠
            , SUM(CASE WHEN 장르번호 = 45                 THEN 1 ELSE 0 END )                 AS 공상과학
            , SUM(CASE WHEN 장르번호 = 46 OR 장르번호 = 53 THEN 1 ELSE 0 END )                 AS 전기
            , SUM(CASE WHEN 장르번호 = 47 OR 장르번호 = 51 THEN 1 ELSE 0 END )                 AS 뉴스
    FROM (
        SELECT 
              a01.회원번호
            , a01.기준년월
            , a01.영화번호
            , a01.행동패턴
            , a02.장르번호
            , a03.장르명
        FROM (
            SELECT
                  회원번호
                , 기준년월
                , 영화번호
                , 구매여부 || 시작여부 || 완료여부 || 평점여부                   AS 행동패턴
            FROM CUST_ACT 
        ) a01
        
        LEFT JOIN MOVIE_GENRE a02
        ON a01.영화번호 = a02.영화번호
        
        LEFT JOIN GENRE a03
        ON a02.장르번호 = a03.장르번호

        WHERE a01.행동패턴 <> '0000' 
   )
    GROUP BY 기준년월
   
    """
, conn
)

,기준년월,역사,애니메이션,드라마,코미디,액션,범죄,스릴러,다큐멘터리,모험,...,공포,전쟁,서부극,미스터리,단극,뮤지컬,스포츠,공상과학,전기,뉴스
0,201001,792,1572,9243,6029,4143,2989,4608,250,2669,...,2849,2316,494,828,837,1121,1267,3601,1922,32
1,201002,634,1125,7169,4504,3137,2334,3592,176,2053,...,2146,1740,355,697,641,828,977,2647,1512,42
2,201003,664,1306,7826,5007,3591,2513,3892,203,2293,...,2453,1945,338,678,706,984,1138,3030,1686,38
3,201004,661,1118,7260,4638,3278,2441,3572,175,2045,...,2169,1744,355,661,685,905,1049,2727,1594,34
4,201005,759,1391,8438,5517,3711,2738,4143,224,2428,...,2533,2170,443,793,802,1061,1210,3182,1970,46
5,201006,699,1232,7565,4845,3471,2544,3836,176,2289,...,2369,1918,391,741,683,932,1094,2909,1684,34
6,201007,681,1280,7948,5042,3518,2627,4077,232,2261,...,2479,1982,404,758,707,990,1057,3022,1820,54
7,201008,686,1284,7923,5040,3446,2524,3915,217,2297,...,2501,2013,414,717,708,961,1144,3064,1746,54
8,201009,686,1264,7642,4919,3388,2530,3830,189,2233,...,2401,1889,392,711,745,888,1064,2969,1692,38
9,201010,744,1403,8784,5599,3984,2902,4426,272,2550,...,2749,2269,460,826,831,1056,1213,3362,1922,44


In [14]:
######################################################################
### 1-3. 복수 장르 보유 영화 추출
######################################################################
df_genre_dup = pd.read_sql(f""" 
     SELECT 
            t1.영화번호
          , t2.영화명
          , t1.장르수
          , t1.목록
          , case when instr(t1.목록, '3,' )  > 0 then 1      else 0 end                           as 장르_드라마
          , case when instr(t1.목록, '6,' )  > 0 then 1      else 0 end                           as 장르_코미디
          , case when instr(t1.목록, '9,' )  > 0 then 1      else 0 end                           as 장르_스릴러
          , case when instr(t1.목록, '15,')  > 0 then 1      else 0 end                           as 장르_로맨스
          , case when instr(t1.목록, '17,')  > 0 then 1      else 0 end                           as 장르_공포
          , case when instr(t1.목록, '7,' )  > 0 then 1      else 0 end                           as 장르_액션
          , case when instr(t1.목록, '45,')  > 0 then 1      else 0 end                           as 장르_공상과학
          , case when instr(t1.목록, '8,' )  > 0 then 1      else 0 end                           as 장르_범죄
          , case when instr(t1.목록, '12,')  > 0 then 1      else 0 end                           as 장르_판타지
          , case when instr(t1.목록, '14,')  > 0 then 1      else 0 end                           as 장르_가족
          , case when instr(t1.목록, '11,')  > 0 then 1      else 0 end                           as 장르_모험
          , case when instr(t1.목록, '18,')  > 0 then 1      else 0 end                           as 장르_전쟁
          , case when instr(t1.목록, '16,')  > 0 then 1      else 0 end                           as 장르_음악
          , case when instr(t1.목록, '25,')  > 0 then 1      else 0 end                           as 장르_뮤지컬
          , case when instr(t1.목록, '30,')  > 0 then 1      else 0 end                           as 장르_스포츠
          , case when instr(t1.목록, '19,')  > 0 then 1      else 0 end                           as 장르_서부극
          , case when instr(t1.목록, '2,' )  > 0 then 1      else 0 end                           as 장르_애니메이션
          , case when instr(t1.목록, '10,')  > 0 then 1      else 0 end                           as 장르_다큐멘터리
          , case when instr(t1.목록, '53,')  > 0 or
                      instr(t1.목록, '46,')  > 0 then 1      else 0 end                           as 장르_전기
          , case when instr(t1.목록, '20,')  > 0 then 1      else 0 end                           as 장르_미스터리
          , case when instr(t1.목록, '24,')  > 0 then 1      else 0 end                           as 장르_단극
          , case when instr(t1.목록, '1,' )  > 0 then 1      else 0 end                           as 장르_역사
          , case when instr(t1.목록, '51,')  > 0 or 
                      instr(t1.목록, '47,')  > 0 then 1      else 0 end                           as 장르_뉴스
     FROM (
          SELECT 
                 영화번호
               , COUNT(장르번호)                                                                  as 장르수
               , GROUP_CONCAT(장르번호) || ','                                                    as 목록
          FROM movie_genre
          GROUP BY 영화번호
     ) t1

     --------------------------------------------------
     -- 영화명 데이터 추출을 위한 조인
     --------------------------------------------------
     LEFT JOIN movie t2
     ON  t1.영화번호 = t2.영화번호

     --------------------------------------------------
     -- 장르수 2개 이상인 영화만 추출
     --------------------------------------------------
     WHERE t1.장르수 > 1 

     GROUP BY t1.영화번호
""" 
, conn) 
df_genre_dup.drop(columns = ['영화번호', '영화명', '장르수', '목록']).sum().to_frame(name = '중복발생건수')


,중복발생건수
장르_드라마,1869
장르_코미디,1607
장르_스릴러,832
장르_로맨스,655
장르_공포,474
장르_액션,967
장르_공상과학,511
장르_범죄,813
장르_판타지,487
장르_가족,419


In [15]:
######################################################################
### 1-4. 장르별 매출총액, 구매건수 탐색
######################################################################
pd.read_sql(
 f""" 
    SELECT
          t1.장르번호 
        , t2.장르명
        , COUNT(1)                      AS 구매건수
        , sum(t1.판매액)                AS 총판매액
    
    FROM movie_fact t1
    
    LEFT JOIN genre t2
    ON t2.장르번호 = t1.장르번호
    
    WHERE t1.판매액 is not null 
        AND t1.장르번호 IN (2, 7, 8, 9, 11, 12, 14, 15, 17, 18, 45)  -- 1차 필터링 이후 11개의 장르
    
    GROUP BY 
        t1.장르번호
    , t2.장르명
        
    ORDER BY 총판매액 DESC
 """ 
, conn) 

,장르번호,장르명,구매건수,총판매액
0,17,Horror,46102,97714.98
1,9,Thriller,37685,91184.15
2,7,Action,31905,69788.95
3,11,Adventure,23448,56848.52
4,8,Crime,21515,52027.85
5,45,Sci-Fi,22615,49689.85
6,18,War,18885,47642.15
7,15,Romance,16734,37983.66
8,12,Fantasy,13700,30946.00
9,14,Family,8954,21068.46


In [16]:
######################################################################
### 1-5. 판매 실적이 낮은 영화
######################################################################
pd.read_sql(
 f""" 
    SELECT 
          a02.장르번호
        , a03.장르명
        , SUM(1)                                        AS 전체건수
        , SUM(CASE WHEN 구매여부 = 1 THEN 1 ELSE 0 END ) AS 구매건수
    FROM (
        SELECT
              영화번호
            , 구매여부
        FROM CUST_ACT 
        WHERE 구매여부 || 시작여부 || 완료여부 || 평점여부 <> '0000'
    ) a01
    
    -- 장르번호 매핑
    LEFT JOIN MOVIE_GENRE a02
    ON a01.영화번호 = a02.영화번호
    
    -- 장르명 매핑
    LEFT JOIN GENRE a03
    ON a02.장르번호 = a03.장르번호

    WHERE a02.장르번호 IN (7, 11, 8)  -- 후보군 3개의 장르
    
    GROUP BY 
          a02.장르번호
        , a03.장르명
    
    ORDER BY SUM(CASE WHEN 구매여부 = 1 THEN 1 ELSE 0 END ) DESC
"""
, conn)

,장르번호,장르명,전체건수,구매건수
0,7,Action,154746,98477
1,8,Crime,112397,71290
2,11,Adventure,100826,65489


In [17]:
pd.read_sql(f"""
    SELECT 
          a01.회원번호
        , a01.기준년월
        , a01.영화번호
        , a01.행동패턴
        , SUBSTR(a01.행동패턴, 4, 1)                                                 AS 평점부여여부
        , a03.장르번호
        , a04.장르명
        
    FROM (
        SELECT
              회원번호
            , 기준년월
            , 영화번호
            , 구매여부 || 시작여부 || 완료여부 || 평점여부                         AS 행동패턴
        FROM CUST_ACT 
    ) a01
    
    -- 장르번호 매핑
    LEFT JOIN MOVIE_GENRE a03
    ON a01.영화번호 = a03.영화번호

    -- 장르명 매핑
    LEFT JOIN GENRE a04
    ON a04.장르번호 = a03.장르번호
    
    WHERE a01.행동패턴 <> '0000' 
        AND a03.장르번호 IN (7,8)
    """
, conn)

,회원번호,기준년월,영화번호,행동패턴,평점부여여부,장르번호,장르명
0,1000693,201001,185,1110,0,8,Crime
1,1000693,201001,605,1111,1,7,Action
2,1000693,201001,769,1111,1,8,Crime
3,1000693,201001,1271,0001,1,7,Action
4,1000693,201001,2114,0001,1,7,Action
...,...,...,...,...,...,...,...
267138,1417816,201210,604,1110,0,7,Action
267139,1417816,201210,36648,1111,1,7,Action
267140,1417816,201210,44758,0001,1,7,Action
267141,1445052,201210,829,1010,0,8,Crime


In [18]:
######################################################################
### 1-6. 2차 장르 제외 3번 
###     - 장르별 최신영화 보유 개수
###     - 장르별 블록버스터 보유 개수
###     - 장르별 평균 평점 비교
######################################################################
# 장르별 최신영화 보유 개수
df_recent = pd.read_sql(f""" 
    select
          t1.*
        , t2.개봉년
        , count(1)                                              AS 최신영화개수
    from movie_genre t1
    
    left join movie t2
    on t1.영화번호 = t2.영화번호
    
    where 장르번호 in (7,8)
       and 개봉년 > 2004 
       and 개봉년 < 2014
       
    group by 장르번호
 """ 
, conn)
display(df_recent)


# 장르별 블록버스터 영화 보유 개수 집계
df_buster = pd.read_sql(f""" 
    select
          t1.*
        , t2.개봉년
        , count(1)                                              AS 블록버스터영화개수
    from movie_genre t1
    
    left join movie t2
    on t1.영화번호 = t2.영화번호
    
    where 장르번호 in (7,8)
    and 박스오피스 > 100000000
    group by 장르번호
 """ 
, conn)
display(df_buster)

# 장르별 평점 집계
df_ratings = pd.read_sql(f""" 
    SELECT 
          T1.장르번호
        , COUNT(1)
        , SUM(T2.평점)                                                                    AS 평점총합
        , AVG(T2.평점)                                                                     AS 평점평균
    FROM(
          SELECT 
                a01.회원번호
              , a01.기준년월
              , a01.영화번호
              , a01.행동패턴
              , SUBSTR(a01.행동패턴, 4, 1)                                                 AS 평점부여여부
              , a03.장르번호
              , a04.장르명
          FROM (
              SELECT
                    회원번호
                  , 기준년월
                  , 영화번호
                  , 구매여부 || 시작여부 || 완료여부 || 평점여부                         AS 행동패턴
              FROM CUST_ACT 
          ) a01
          
          -- 장르번호 매핑
          LEFT JOIN MOVIE_GENRE a03
          ON a01.영화번호 = a03.영화번호

          -- 장르명 매핑
          LEFT JOIN GENRE a04
          ON a04.장르번호 = a03.장르번호
          
          WHERE a01.행동패턴 <> '0000' 
             AND a03.장르번호 IN (7,8)
    ) T1
    
    LEFT JOIN (
            SELECT 
                  기준년월
                , 영화번호
                , 회원번호
                , 평점
            FROM MOVIE_FACT 
            WHERE 평점 IS NOT NULL
    ) T2
    ON  
        T2.기준년월 = T1.기준년월
    AND T2.영화번호 = T1.영화번호
    AND T2.회원번호 = T1.회원번호
    
    WHERE T1.평점부여여부 = '1'
    AND   T1.장르번호 IN (7,8)
    
    GROUP BY T1.장르번호
      
 """ 
, conn) 
display(df_ratings)


,영화번호,장르번호,개봉년,최신영화개수
0,47808,7,2005,203
1,43539,8,2010,136


,영화번호,장르번호,개봉년,블록버스터영화개수
0,1022872,7,1994,258
1,20174,8,2001,92


,장르번호,COUNT(1),평점총합,평점평균
0,7,67976,289327.0,4.256311
1,8,50011,220183.0,4.402691
